### Classifying the images based on various disease of images

In [11]:
# !pip install timm

DEPRECATION: Loading egg at /Users/mac/anaconda3/lib/python3.11/site-packages/python_docx-1.1.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/mac/anaconda3/lib/python3.11/site-packages/Pattern-3.6.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached timm-1.0.8-py3-none-any.whl.metadata (53 kB)
Using cached timm-1.0.8-py3-none-any.whl (2.3 MB)
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency spec

In [24]:
import timm

In [12]:
models_path = {
    'Vit-Model': 'models/vit_base_patch16_224_best.pth',
    'Efficientnet': 'models/efficientnet_b0_best.pth',
    'Densenet': 'models/densenet121_best.pth',
    'Resnet50': 'models/resnet50_best.pth'
}

In [20]:
# Define the classes
classes = [
    "Diabetes",
    "Normal Fundus",
    "Cataract"
]


def classify(model, model_name, image_path, classes):
    # Define the mean and std for normalization
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    # Define image transformations
    image_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
    ])

    model = model.eval()
    from PIL import Image

    image = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB mode
    image = image_transforms(image).float()
    image = image.unsqueeze(0)  # Add batch dimension

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    image = image.to(device)

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)

    print(f"{model_name} -> ", classes[predicted.item()])


In [47]:
def load_vit_model():
    # Initialize the Vision Transformer (ViT) model architecture
    vit_model = timm.create_model('vit_base_patch16_224', pretrained=False)
    num_classes = len(classes)
    vit_model.head = torch.nn.Linear(vit_model.head.in_features, num_classes)

    # Load the saved state dictionary
    vit_model.load_state_dict(torch.load(models_path.get('Vit-Model'), map_location=torch.device('cpu')))

    print('Vit-Model loaded successfully')
    return vit_model

In [48]:
vit_model = load_vit_model()

Vit-Model loaded successfully


In [8]:
def load_efficientnet_model():
    # Initialize the EfficientNet-B0 model architecture
    efficientnet_model = models.efficientnet_b0(pretrained=False)
    num_classes = len(classes)
    efficientnet_model.classifier[1] = torch.nn.Linear(efficientnet_model.classifier[1].in_features, num_classes)

    # Load the saved state dictionary
    efficientnet_model.load_state_dict(torch.load(models_path.get('Efficientnet'), map_location=torch.device('cpu')))

    print('Efficientnet loaded successfully')
    return efficientnet_model

In [50]:
efficientnet_model = load_efficientnet_model()

Efficientnet loaded successfully


In [51]:
def load_densenet_model():
    # Initialize the DenseNet model architecture
    densenet_model = models.densenet121(pretrained=False)
    num_classes = len(classes)
    densenet_model.classifier = torch.nn.Linear(densenet_model.classifier.in_features, num_classes)

    # Load the saved state dictionary
    densenet_model.load_state_dict(torch.load(models_path.get('Densenet'), map_location=torch.device('cpu')))

    print('Densenet loaded successfully')
    return densenet_model

In [52]:
densenet_model = load_densenet_model()

Densenet loaded successfully


In [16]:



def load_resnet_model():
    # Initialize the ResNet-50 model architecture
    resnet_model = models.resnet50(pretrained=False)
    num_classes = len(classes)
    resnet_model.fc = torch.nn.Linear(resnet_model.fc.in_features, num_classes)

    # Load the saved state dictionary
    resnet_model.load_state_dict(torch.load(models_path.get('Resnet50'), map_location=torch.device('cpu')))

    print('Resnet50 loaded successfully')
    return resnet_model

In [17]:
resnet_model = load_resnet_model()

Resnet50 loaded successfully


In [21]:
image_path = "./images/diabetes_1.jpg"
# classify(densenet_model, "Densenet", image_path, classes)
# classify(efficientnet_model, "Efficientnet", image_path, classes)
# classify(vit_model, "Vit-Model", image_path, classes)
classify(resnet_model, "Resnet50", image_path, classes)

Resnet50 ->  Normal Fundus


In [34]:
image_path = "./images/fundus_2.jpg"
classify(densenet_model, "Densenet", image_path, classes)
classify(efficientnet_model, "Efficientnet", image_path, classes)
classify(vit_model, "Vit-Model", image_path, classes)
classify(resnet_model, "Resnet50", image_path, classes)

Densenet ->  Cataract
Efficientnet ->  Normal Fundus
Vit-Model ->  Cataract
Resnet50 ->  Normal Fundus


In [23]:
%%writefile app.py
import streamlit as st
import torch
from PIL import Image
from torchvision import transforms, models

# Define the classes
classes = [
    "Cataract",
    "Diabetes",
    "Glaucoma",
]


# @st.cache_resource
def load_resnet_model():
    # Initialize the ResNet-50 model architecture
    resnet_model = models.resnet50(pretrained=False)
    num_classes = len(classes)
    resnet_model.fc = torch.nn.Linear(resnet_model.fc.in_features, num_classes)

    # Load the saved state dictionary
    resnet_model.load_state_dict(torch.load('models/resnet50_best.pth', map_location=torch.device('cpu')))

    print('Resnet50 loaded successfully')
    return resnet_model


# Load the model
resnet_model = load_resnet_model()()


def classify_eye(model, image):
    model = model.eval()

    # Define image transformations
    image_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    # Preprocess the image
    image = image.convert('RGB')  # Ensure the image is in RGB mode
    image = image_transforms(image).unsqueeze(0)  # Add batch dimension

    # Move model and image to the appropriate device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    image = image.to(device)

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)

    return classes[predicted.item()]


st.title("Eye Image Classification")

uploaded_file = st.file_uploader("Upload an image of an eye", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)
    st.write("")

    if st.button('Classify Image'):
        result = classify_eye(resnet_model, image)
        st.write(f"Prediction: {result}")

Overwriting app.py


In [64]:
# !streamlit run app.py